In [0]:
###############################
#                             #
#        ENCODE FACTORS       #
#                             #
###############################

# performs label encoding
from sklearn.preprocessing import LabelEncoder
def label_encoding(df_train, df_valid, df_test):
    
    factors = df_train.select_dtypes('object').columns
    
    lbl = LabelEncoder()

    for f in factors:        
        lbl.fit(list(df_train[f].values) + list(df_valid[f].values) + list(df_test[f].values))
        df_train[f] = lbl.transform(list(df_train[f].values))
        df_valid[f] = lbl.transform(list(df_valid[f].values))
        df_test[f]  = lbl.transform(list(df_test[f].values))

    return df_train, df_valid, df_test

import numpy as np
def reduce_mem_usage(df, verbose = True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: 
        print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df


In [2]:
# GOOGLE COLAB SETUP

# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
os.chdir('drive/My Drive/Colab Notebooks/WIDS/WIDS/')

In [0]:

import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('dark_background')
%matplotlib inline

import os
import time
import datetime
import random
import multiprocessing
import pickle

import scipy.stats

import gc
gc.enable()

import warnings
warnings.filterwarnings('ignore')


from sklearn.model_selection import KFold, StratifiedKFold, GroupKFold
from sklearn.preprocessing import RobustScaler, LabelEncoder
from sklearn.metrics import log_loss, roc_auc_score, confusion_matrix

import lightgbm as lgb

In [0]:
import sys
sys.path.append('drive/My Drive/Colab Notebooks/WIDS/WIDS/codes')

In [0]:
############ RANDOMNESS

# seed function
def seed_everything(seed = 42):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    
# set seed
seed = 42
seed_everything(seed)

### IMPORT

In [7]:
############ DATA IMPORT

# id data
train = pd.read_csv('./raw/training_v2.csv')
test  = pd.read_csv('./raw/unlabeled.csv')


# check dimensions
print(train.shape)
print(train.shape)

train = train[-train['hospital_death'].isnull()]

(91713, 186)
(91713, 186)


In [0]:
y     = train['hospital_death']
train = train.drop('hospital_death', axis=1)

In [0]:
############ FEAUTERS

# drop bad features
excluded_feats = ['encounter_id', 'patient_id', 'readmission_status', 'hospital_id', 'icu_id']

In [0]:
train['hospital_id'] = train['hospital_id'].astype('object')
test['hospital_id']  = test['hospital_id'].astype('object')

In [11]:
features = [f for f in train.columns if f not in excluded_feats]
print(train[features].shape)

(91713, 180)


In [0]:
############ PARAMETERS

# cores
cores = -1
# cross-validation
num_folds = 10
shuffle   = True

# number of trees
max_rounds = 10000
stopping   = 600
verbose    = 250

# LGB parameters
lgb_params = {
    'boosting_type':     'gbdt',
    'objective':         'binary',
    'metric':            'auc',
    'bagging_fraction':  0.9,
    'feature_fraction':  0.9,
    'lambda_l1':         0.1,
    'lambda_l2':         0.1,
    'min_split_gain':    0,
    'min_child_weight':  0.1,
    'min_child_samples': 10,
    'silent':            True,
    'verbosity':         -1,
    'learning_rate':     0.01,
    'max_depth':         5,
    'num_leaves':        64,
    'scale_pos_weight':  1,
    'n_estimators':      max_rounds,
    'nthread' :          cores,
    'random_state':      seed,
    #"device" : "gpu"
}

# data partitinoing
folds = StratifiedKFold(n_splits = num_folds, random_state = seed, shuffle = shuffle)
#folds = GroupKFold(n_splits = num_folds)
#folds = model_selection.TimeSeriesSplit(n_splits = 10)

# SMOTE settings
#from imblearn.over_sampling import SMOTE
#sm = SMOTE(random_state = seed, n_jobs = cores, sampling_strategy = 0.05)

In [0]:

############ PLACEHOLDERS

# placeholders
clfs = []
importances = pd.DataFrame()

# predictions
preds_test   = np.zeros(test.shape[0])
preds_oof    = np.zeros(train.shape[0])

In [0]:
train.fillna(train.median(), inplace=True)
test.fillna(train.median(), inplace=True)

In [0]:
############ CROSS-VALIDATION LOOP
cv_start  = time.time()
for n_fold, (trn_idx, val_idx) in enumerate(folds.split(train, y)):

    # data partitioning
    trn_x, trn_y = train[features].iloc[trn_idx], y.iloc[trn_idx]
    val_x, val_y = train[features].iloc[val_idx], y.iloc[val_idx]
    test_x       = test[features]
    
    ## augment training data with SMOTE
    #trn_x[trn_x.columns]  = trn_x[trn_x.columns].apply(pd.to_numeric,   errors = 'coerce')
    #val_x[val_x.columns]  = val_x[val_x.columns].apply(pd.to_numeric,   errors = 'coerce')
    #test_x[val_x.columns] = test_x[test_x.columns].apply(pd.to_numeric, errors = 'coerce')
    #trn_x  = trn_x.replace([np.inf,  -np.inf], np.nan)
    #val_x  = val_x.replace([np.inf,  -np.inf], np.nan)
    #test_x = test_x.replace([np.inf, -np.inf], np.nan)
    #trn_x  = trn_x.fillna(trn_x.median())
    #val_x  = val_x.fillna(val_x.median())
    #test_x = test_x.fillna(test_x.median())
    #trn_x, trn_y = sm.fit_sample(trn_x, trn_y)
    #trn_x = pd.DataFrame(trn_x, columns = features)
    #trn_y = pd.Series(trn_y)
    
    # Aggregate data
    '''agg_features = list(trn_x.select_dtypes('number').columns)
    aggregated = list(trn_x.select_dtypes('object').columns)
    for variable in aggregated:
        print(f'preparing {variable}')
        agg_features.append(variable)
        features_ = trn_x[agg_features].groupby(variable).mean()
        features_.columns = [f'{variable}_{column}_mean' for column in features_.columns]
        trn_x  = trn_x.merge(features_, left_on=variable, right_on=variable, how='left')
        val_x  = val_x.merge(features_, left_on=variable, right_on=variable, how='left')
        test_x = test_x.merge(features_, left_on=variable, right_on=variable, how='left')
        agg_features.remove(variable)'''
        
    '''for variable in aggregated:
        print(f'preparing {variable}')
        agg_features.append(variable)
        features_ = trn_x[agg_features].groupby(variable).sum()
        features_.columns = [f'{variable}_{column}_sum' for column in features_.columns]
        trn_x  = trn_x.merge(features_, left_on=variable, right_on=variable, how='left')
        val_x  = val_x.merge(features_, left_on=variable, right_on=variable, how='left')
        test_x = test_x.merge(features_, left_on=variable, right_on=variable, how='left')
        agg_features.remove(variable)
    
    for variable in aggregated:
        print(f'preparing {variable}')
        agg_features.append(variable)
        features_ = trn_x[agg_features].groupby(variable).std()
        features_.columns = [f'{variable}_{column}_std' for column in features_.columns]
        trn_x  = trn_x.merge(features_, left_on=variable, right_on=variable, how='left')
        val_x  = val_x.merge(features_, left_on=variable, right_on=variable, how='left')
        test_x = test_x.merge(features_, left_on=variable, right_on=variable, how='left')
        agg_features.remove(variable)'''
        
        
    # Fill Na
    trn_x['weight']  = trn_x['weight'].fillna(trn_x.groupby(['ethnicity','age','gender'])['weight'].transform('mean'))
    val_x['weight']  = val_x['weight'].fillna(trn_x.groupby(['ethnicity','age','gender'])['weight'].transform('mean'))
    test_x['weight'] = test_x['weight'].fillna(trn_x.groupby(['ethnicity','age','gender'])['weight'].transform('mean'))
    
    trn_x['height']  = trn_x['height'].fillna(trn_x.groupby(['ethnicity','age','gender'])['height'].transform('mean'))
    val_x['height']  = val_x['height'].fillna(trn_x.groupby(['ethnicity','age','gender'])['height'].transform('mean'))
    test_x['height'] = test_x['height'].fillna(trn_x.groupby(['ethnicity','age','gender'])['height'].transform('mean'))
    
    trn_x['bmi']  = trn_x['bmi'].fillna(trn_x.groupby(['ethnicity','age','gender'])['bmi'].transform('mean'))
    val_x['bmi']  = val_x['bmi'].fillna(trn_x.groupby(['ethnicity','age','gender'])['bmi'].transform('mean'))
    test_x['bmi'] = test_x['bmi'].fillna(trn_x.groupby(['ethnicity','age','gender'])['bmi'].transform('mean'))
    
    for column in trn_x.select_dtypes('object').columns:
        trn_x[column] = trn_x[column].fillna('')
        val_x[column] = val_x[column].fillna('')
        test_x[column] = test_x[column].fillna('')
        
    # label encoding
    trn_x, val_x, test_x = label_encoding(trn_x, val_x, test_x)
    
    
    ## remove outliers
    # num_vars = trn_x.select_dtypes(include='number')
    #num_vars = num_vars.columns
    #for num_var in num_vars:
    #    trn_x[num_var] = trn_x[num_var].replace([np.inf, -np.inf], np.nan)
    #    trn_x[num_var] = trn_x[num_var].fillna(trn_x[num_var].median())
    #out_idx = (np.abs(scipy.stats.zscore(trn_x[num_vars])) < 20).all(axis = 1) + (trn_y.values == 1)
    #trn_x = trn_x[out_idx]
    
    ## scale data
    #val_x  = val_x.replace([np.inf, -np.inf], np.nan)
    
    #scaler   = RobustScaler()
    #trn_x    = pd.DataFrame(scaler.fit_transform(trn_x))
    #val_x    = pd.DataFrame(scaler.transform(val_x))
    #test_x   = pd.DataFrame(scaler.transform(test_x))
       
    ## add noise to train to reduce overfitting
    trn_x += np.random.normal(0, 0.01, trn_x.shape)
    
    # print data dimensions
    print('Data shape:', trn_x.shape, val_x.shape)
    #print('Data shape:', trn_y.shape, val_y.shape)    
    # train lightGBM
    clf = lgb.LGBMClassifier(**lgb_params) 
    clf = clf.fit(trn_x, trn_y, 
                  eval_set              = [(trn_x, trn_y), (val_x, val_y)], 
                  eval_metric           = 'auc', 
                  early_stopping_rounds = stopping,
                  verbose               = verbose)
    clfs.append(clf)
    
    # find the best iteration
    best_iter = clf.best_iteration_

    # save predictions
    preds_oof[val_idx] = clf.predict_proba(val_x,  num_iteration = best_iter)[:, 1]
    preds_test        += clf.predict_proba(test_x, num_iteration = best_iter)[:, 1] / folds.n_splits 

    # importance
    fold_importance_df               = pd.DataFrame()
    fold_importance_df['Feature']    = trn_x.columns
    fold_importance_df['Importance'] = clf.feature_importances_
    fold_importance_df['Fold']       = n_fold + 1
    importances                      = pd.concat([importances, fold_importance_df], axis = 0)
    
    # print performance
    print('--------------------------------')
    print('FOLD%2d: AUC = %.6f' % (n_fold + 1, roc_auc_score(y[val_idx], preds_oof[val_idx])))
    print('--------------------------------')
    print('')
        
    # clear memory
    del trn_x, trn_y, val_x, val_y
    gc.collect()
    
    
# print overall performance    
cv_perf = roc_auc_score(y, preds_oof)
print('--------------------------------')
print('- OOF AUC = %.6f' % cv_perf)
print('- CV TIME = {:.2f} min'.format((time.time() - cv_start) / 60))
print('--------------------------------')

Data shape: (82541, 180) (9172, 180)
Training until validation scores don't improve for 600 rounds.
[250]	training's auc: 0.909104	valid_1's auc: 0.888327
[500]	training's auc: 0.925679	valid_1's auc: 0.896576
[750]	training's auc: 0.935998	valid_1's auc: 0.898795
[1000]	training's auc: 0.943977	valid_1's auc: 0.900015
[1250]	training's auc: 0.950956	valid_1's auc: 0.900863
[1500]	training's auc: 0.957085	valid_1's auc: 0.901945
[1750]	training's auc: 0.962425	valid_1's auc: 0.902667
[2000]	training's auc: 0.966805	valid_1's auc: 0.902897
[2250]	training's auc: 0.970777	valid_1's auc: 0.903184
[2500]	training's auc: 0.974105	valid_1's auc: 0.903301
[2750]	training's auc: 0.977167	valid_1's auc: 0.903354
[3000]	training's auc: 0.979875	valid_1's auc: 0.903262
[3250]	training's auc: 0.982387	valid_1's auc: 0.903271
Early stopping, best iteration is:
[2820]	training's auc: 0.977986	valid_1's auc: 0.903439
--------------------------------
FOLD 1: AUC = 0.903439
----------------------------

KeyboardInterrupt: ignored

### EVALUATION

In [0]:
############ RECHECK PERFORMANCE  

# check performance
print(np.round(roc_auc_score(y, preds_oof), 5))


############ TRACK RESULTS

In [0]:
############ VARIABLE IMPORTANCE

# load importance    
top_feats = 300
cols = importances[['Feature', 'Importance']].groupby('Feature').mean().sort_values(by = 'Importance', ascending = False)[0:top_feats].index
importance = importances.loc[importances.Feature.isin(cols)]
    
# plot variable importance
plt.figure(figsize = (10, 150))
sns.barplot(x = 'Importance', y = 'Feature', data = importance.sort_values(by = 'Importance', ascending = False))
plt.tight_layout()
plt.savefig('../var_importance.pdf')

SUBMISSION

In [0]:
# file name
model = 'lgb_v21'
perf  = str(round(cv_perf, 6))[2:7]
name  = model + '_' + perf
name

In [0]:
# export OOF preds
oof = pd.DataFrame({'encounter_id': train['encounter_id'], 'hospital_death': preds_oof})
oof.to_csv('./oof_preds/' + str(name) + '.csv', index = False)
oof.head()

In [0]:

# export submission
sub = pd.DataFrame({'encounter_id': test['encounter_id'], 'hospital_death': preds_test})
sub.to_csv('./submissions/' + str(name) + '.csv', index = False)
sub.head()